## 1. Постановка задачи

<center> <img src=https://storage.googleapis.com/kaggle-competitions/kaggle/3333/media/taxi_meter.png align="right" width="300"/> </center>
    
Вам предстоит решить настоящую задачу машинного обучения, направленную на автоматизацию бизнес процессов. Мы построим модель, которая будет предсказывать общую продолжительность поездки такси в Нью-Йорке. 

Представьте вы заказываете такси из одной точки Нью-Йорка в другую, причем не обязательно конечная точка должна находиться в пределах города. Сколько вы должны будете за нее заплатить? Известно, что стоимость такси в США  рассчитывается на основе фиксированной ставки + тарифная стоимость, величина которой зависит от времени и расстояния. Тарифы варьируются в зависимости от города.

В свою очередь время поездки зависит от множества факторов таких как, откуда и куда вы едете, в какое время суток вы совершаете вашу поездку, погодных условий и так далее. 

Таким образом, если мы разработаем алгоритм, способный определять длительность поездки, мы сможем прогнозировать ее стоимость самым тривиальным образом, например, просто умножая стоимость на заданный тариф. 
Сервисы такси хранят огромные объёмы информации о поездках, включая такие данные как конечная, начальная точка маршрута, дата поездки и ее длительность. Эти данные можно использовать для того, чтобы прогнозировать длительность поездки в автоматическом режиме с привлечением искусственного интеллекта.

**Бизнес-задача:** определить характеристики и с их помощью спрогнозировать длительность поездки такси.

**Техническая задача для вас как для специалиста в Data Science:** построить модель машинного обучения, которая на основе предложенных характеристик клиента будет предсказывать числовой признак - время поездки такси. То есть решить задачу регрессии.

**Основные цели проекта:**
1. Сформировать набор данных на основе нескольких источников информации
2. Спроектировать новые признаки с помощью Feature Engineering и выявить наиболее значимые при построении модели
3. Исследовать предоставленные данные и выявить закономерности
4. Построить несколько моделей и выбрать из них наилучшую по заданной метрике
5. Спроектировать процесс предсказания времени длительности поездки для новых данных

Загрузить свое решение на платформу Kaggle, тем самым поучаствовав в настоящем Data Science соревновании.
Во время выполнения проекта вы отработаете навыки работы с несколькими источниками данных, генерации признаков, разведывательного анализа и визуализации данных, отбора признаков и, конечно же, построения моделей машинного обучения!


## 2. Знакомство с данными, базовый анализ и расширение данных

Начнём наше исследование со знакомства с предоставленными данными. А также подгрузим дополнительные источники данных и расширим наш исходный датасет. 


Заранее импортируем модули, которые нам понадобятся для решения задачи:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

Прочитаем наш файл с исходными данными:

In [4]:
taxi_data = pd.read_csv("data/train.csv")
print('Train data shape: {}'.format(taxi_data.shape))
taxi_data.head()

Train data shape: (1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


Итак, у нас с вами есть данные о почти 1.5 миллионах поездок и 11 характеристиках, которые описывают каждую из поездок. 

Мы условно разделили признаки нескольких групп. Каждой из групп мы в дальнейшем уделим отдельное внимание.

**Данные о клиенте и таксопарке:**
* id - уникальный идентификатор поездки
* vendor_id - уникальный идентификатор поставщика (таксопарка), связанного с записью поездки

**Временные характеристики:**
* pickup_datetime - дата и время, когда был включен счетчик поездки
* dropoff_datetime - дата и время, когда счетчик был отключен

**Географическая информация:**
* pickup_longitude -  долгота, на которой был включен счетчик
* pickup_latitude - широта, на которой был включен счетчик
* dropoff_longitude - долгота, на которой счетчик был отключен
* dropoff_latitude - широта, на которой счетчик был отключен

**Прочие признаки:**
* passenger_count - количество пассажиров в транспортном средстве (введенное водителем значение)
* store_and_fwd_flag - флаг, который указывает, сохранилась ли запись о поездке в памяти транспортного средства перед отправкой поставщику. Y - хранить и пересылать, N - не хранить и не пересылать поездку.

**Целевой признак:**
* trip_duration - продолжительность поездки в секундах


Для начала мы проведем базовый анализ того, насколько данные готовы к дальнейшей предобработке и анализу. 

### Задание 2.1
Для начала посмотрим на временные рамки, в которых мы работаем с данными.

Переведите признак pickup_datetime в тип данных datetime с форматом год-месяц-день час:минута:секунда (в функции pd.to_datetime() параметр format='%Y-%m-%d %H:%M:%S'). 

Определите временные рамки (без учета времени), за которые представлены данные.

In [5]:
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

print(f'Данные предоставлены за период с {taxi_data.pickup_datetime.min().date()} по {taxi_data.pickup_datetime.max().date()}')

Данные предоставлены за период с 2016-01-01 по 2016-06-30


### Задание 2.2
Посмотрим на пропуски. 
Сколько пропущенных значений присутствует в данных (суммарно по всем столбцам таблицы)?

In [6]:
print('Количество пропусков данных: ', taxi_data.isna().sum().sum())

Количество пропусков данных:  0


### Задание 2.3
Посмотрим на статистические характеристики некоторых признаков. 

а) Сколько уникальных таксопарков присутствует в данных?

б) Каково максимальное количество пассажиров?

в) Чему равна средняя и медианная длительность поездки? Ответ приведите в секундах и округлите до целого.

г) Чему равно минимальное и максимальное время поездки (в секундах)?


In [7]:
print(f'Количество уникальных таксопарков: {len(taxi_data.vendor_id.unique())}')
print(f'Максимальное количество пассажиров: {taxi_data.passenger_count.max()}')
print(f'Длительность поездки средняя: {round(taxi_data.trip_duration.mean())} и медианная: {round(taxi_data.trip_duration.median())}')
print(f'Длительность поездки минимальная: {taxi_data.trip_duration.min()} и максимальная: {taxi_data.trip_duration.max()}')

Количество уникальных таксопарков: 2
Максимальное количество пассажиров: 9
Длительность поездки средняя: 959 и медианная: 662
Длительность поездки минимальная: 1 и максимальная: 3526282


Займемся расширением исходного набора данных как с помощью внешних источников, так и с помощью манипуляций над имеющимися в данных признаками. 


### Задание 2.4
Реализуйте функцию add_datetime_features(), которая принимает на вход таблицу с данными о поездках (DataFrame) и возвращает ту же таблицу с добавленными в нее 3 столбцами:
* pickup_date - дата включения счетчика - начала поездки (без времени);
* pickup_hour - час дня включения счетчика;
* pickup_day_of_week - наименование дня недели, в который был включен счетчик.

а) Сколько поездок было совершено в субботу?

б) Сколько поездок в среднем совершается в день? Ответ округлите до целого

In [8]:
def add_datetime_features(data):
    data['pickup_date'] = data['pickup_datetime'].dt.date
    data['pickup_hour'] = data['pickup_datetime'].dt.hour
    data['pickup_day_of_week'] = data['pickup_datetime'].dt.day_name ()
    return data

taxi_data = add_datetime_features(taxi_data)

print('Количество поездок в субботу: ', taxi_data['pickup_day_of_week'][taxi_data['pickup_day_of_week'] == 'Saturday'].shape[0])
print('Среднее количество поездок в день: ', round(taxi_data['pickup_date'].value_counts().mean()))


Количество поездок в субботу:  220868
Среднее количество поездок в день:  8015


### Задание 2.5
Реализуйте функцию add_holiday_features(), которая принимает на вход две таблицы: 
* таблицу с данными о поездках;
* таблицу с данными о праздничных днях;

и возвращает обновленную таблицу с данными о поездках с добавленным в нее столбцом pickup_holiday - бинарным признаком того, начата ли поездка в праздничный день или нет (1 - да, 0 - нет). 

Чему равна медианная длительность поездки на такси в праздничные дни? Ответ приведите в секундах, округлив до целого.


In [9]:
holiday_data = pd.read_csv('data/holiday_data.csv', sep=';')

def add_holiday_features(data, holiday):
    holiday['date'] = pd.to_datetime(holiday['date']).dt.date
    holidays = set(holiday['date'])
    data['pickup_holiday'] = data['pickup_date'].apply(lambda x: 1 if x in holidays else 0)
    return data

taxi_data = add_holiday_features(taxi_data, holiday_data)

print('Медианная длительность поездки на такси в праздничные дни: ', round(taxi_data['trip_duration'][taxi_data['pickup_holiday'] == 1].median()))



Медианная длительность поездки на такси в праздничные дни:  585


### Задание 2.6
Реализуйте функцию add_osrm_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными из OSRM;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 3 столбцами:
* total_distance;
* total_travel_time;
* number_of_steps.

а) Чему равна разница (в секундах) между медианной длительностью поездки в данных и медианной длительностью поездки, полученной из OSRM? 

В результате объединения таблиц у вас должны были получиться пропуски в столбцах с информацией из OSRM API. Это связано с тем, что для некоторых поездок не удалось выгрузить данные из веб источника. 

б) Сколько пропусков содержится в столбцах с информацией из OSRM API после объединения таблиц?

In [10]:

osrm_data = pd.read_csv('data/osrm_data_train.csv')

def add_osrm_features(data, osrm):
    return data.merge(osrm[['id', 'total_distance', 'total_travel_time', 'number_of_steps']], on='id', how='left')

taxi_data = add_osrm_features(taxi_data, osrm_data)

distance_diff = round(abs(taxi_data['trip_duration'].median() - taxi_data['total_travel_time'].median()))
print(f'Рразница между медианной длительностью поездки в данных и полученной из OSRM: {distance_diff} сек')

osrm_null = taxi_data[['total_distance', 'total_travel_time', 'number_of_steps']].isna().sum()[0]
print(f'Количество строк с пропусками в признаках из OSRM: {osrm_null}')

Рразница между медианной длительностью поездки в данных и полученной из OSRM: 372 сек
Количество строк с пропусками в признаках из OSRM: 1


In [11]:
def get_haversine_distance(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # радиус земли в километрах
    EARTH_RADIUS = 6371 
    # считаем кратчайшее расстояние h по формуле Хаверсина
    lat_delta = lat2 - lat1
    lng_delta = lng2 - lng1
    d = np.sin(lat_delta * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng_delta * 0.5) ** 2
    h = 2 * EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def get_angle_direction(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # считаем угол направления движения alpha по формуле угла пеленга
    lng_delta_rad = lng2 - lng1
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    alpha = np.degrees(np.arctan2(y, x))
    return alpha

### Задание 2.7.
Реализуйте функцию add_geographical_features(), которая принимает на вход таблицу с данными о поездках и возвращает обновленную таблицу с добавленными в нее 2 столбцами:
* haversine_distance - расстояние Хаверсина между точкой, в которой был включен счетчик, и точкой, в которой счетчик был выключен;
* direction - направление движения из точки, в которой был включен счетчик, в точку, в которой счетчик был выключен.

Чему равно медианное расстояние Хаверсина поездок (в киллометрах)? Ответ округлите до сотых.


In [12]:
def add_geographical_features(data):
    lat1 = data['pickup_latitude'].to_numpy()
    lat2 = data['dropoff_latitude'].to_numpy()
    lng1 = data['pickup_longitude'].to_numpy()
    lng2 = data['dropoff_longitude'].to_numpy()
    data['haversine_distance'] = get_haversine_distance(lat1, lng1, lat2, lng2)
    data['direction'] = get_angle_direction(lat1, lng1, lat2, lng2)
    return data

taxi_data = add_geographical_features(taxi_data)

print(f'Медианное расстояние Хаверсина поездок: {round(taxi_data.haversine_distance.median(), 2)} км')

Медианное расстояние Хаверсина поездок: 2.09 км


### Задание 2.8.
Реализуйте функцию add_cluster_features(), которая принимает на вход таблицу с данными о поездках и обученный алгоритм кластеризации. Функция должна возвращать обновленную таблицу с добавленными в нее столбцом geo_cluster - географический кластер, к которому относится поездка.

Сколько поездок содержится в наименьшем по размеру географическом кластере?


In [13]:
# создаем обучающую выборку из географических координат всех точек
coords = np.hstack((taxi_data[['pickup_latitude', 'pickup_longitude']],
                    taxi_data[['dropoff_latitude', 'dropoff_longitude']]))
# обучаем алгоритм кластеризации
kmeans = cluster.KMeans(n_clusters=10, random_state=42)
kmeans.fit(coords)

def add_cluster_features(data, model):
    X = data[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].to_numpy()
    data['geo_cluster'] = model.predict(X)
    return data

taxi_data = add_cluster_features(taxi_data, kmeans)

print(f'В наименьшем по размеру географическом кластере содержится {taxi_data["geo_cluster"].value_counts().min()} поездки')

# проверяем получившиеся кластеры
taxi_data['geo_cluster'].value_counts().sort_values(ascending = False)

c:\Users\artem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


В наименьшем по размеру географическом кластере содержится 2 поездки


1    592117
0    392108
6    340489
8     45518
4     40234
5     32799
7     15355
9        18
3         4
2         2
Name: geo_cluster, dtype: int64

### Задание 2.9.
Реализуйте функцию add_weather_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными о погодных условиях на каждый час;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 5 столбцами:
* temperature - температура;
* visibility - видимость;
* wind speed - средняя скорость ветра;
* precip - количество осадков;
* events - погодные явления.

а) Сколько поездок было совершено в снежную погоду?

В результате объединения у вас должны получиться записи, для которых в столбцах temperature, visibility, wind speed, precip, и events будут пропуски. Это связано с тем, что в таблице с данными о погодных условиях отсутствуют измерения для некоторых моментов времени, в которых включался счетчик поездки. 

б) Сколько процентов от общего количества наблюдений в таблице с данными о поездках занимают пропуски в столбцах с погодными условиями? Ответ приведите с точностью до сотых процента.


In [14]:
weather_data = pd.read_csv('data/weather_data.csv')

def add_weather_features(data, weather):
    weather['time'] = pd.to_datetime(weather['time'], format='%Y-%m-%d %H:%M:%S')
    weather['date'] = weather['time'].dt.date
    weather['hour'] = weather['time'].dt.hour
    data = data.merge(weather[['date', 'hour', 'temperature', 'visibility', 'wind speed', 'precip', 'events']], 
                      left_on=['pickup_date', 'pickup_hour'], 
                      right_on=['date', 'hour'], 
                      how='left')
    data.drop(['date', 'hour'], axis=1, inplace=True)
    return data

taxi_data = add_weather_features(taxi_data, weather_data)

print(f'В снежную погоду совершено {taxi_data[taxi_data["events"] == "Snow"].shape[0]} поездок')

weather_null = (taxi_data[['temperature', 'visibility', 'wind speed', 'precip', 'events']].isna().sum(axis=1) > 0).sum()
percent_weather_null = round(weather_null / taxi_data.shape[0] * 100, 2)
print(f'Пропуски в столбцах с погодными условиями занимают {percent_weather_null} % от общего количества наблюдений')

В снежную погоду совершено 13126 поездок
Пропуски в столбцах с погодными условиями занимают 0.82 % от общего количества наблюдений


### Задание 2.10.
Реализуйте функцию fill_null_weather_data(), которая принимает на вход таблицу с данными о поездках. Функция должна заполнять пропущенные значения в столбцах.

Пропуски в столбцах с погодными условиями -  temperature, visibility, wind speed, precip заполните медианным значением температуры, влажности, скорости ветра и видимости в зависимости от даты начала поездки. Для этого сгруппируйте данные по столбцу pickup_date и рассчитайте медиану в каждой группе, после чего с помощью комбинации методов transform() и fillna() заполните пропуски. 
Пропуски в столбце events заполните строкой 'None' - символом отсутствия погодных явлений (снега/дождя/тумана). 

Пропуски в столбцах с информацией из OSRM API - total_distance, total_travel_time и number_of_steps заполните медианным значением по столбцам. 

Чему равна медиана в столбце temperature после заполнения пропусков? Ответ округлите до десятых.


In [15]:
def fill_null_weather_data(data):
    col_weather = ['temperature', 'visibility', 'wind speed', 'precip']
    data[col_weather] = data[col_weather].fillna(data.groupby('pickup_date')[col_weather].transform('median'))
    data['events'] = data['events'].fillna('None')
    col_osrm = ['total_distance', 'total_travel_time', 'number_of_steps']
    data[col_osrm] = data[col_osrm].fillna(data[col_osrm].median())
    return data

taxi_data = fill_null_weather_data(taxi_data)

print(f'Медиана в столбце temperature: {taxi_data["temperature"].median()} град.')

Медиана в столбце temperature: 11.1 град.


В завершение первой части найдем очевидные выбросы в целевой переменной - длительности поездки. 

Проще всего найти слишком продолжительные поездки. Давайте условимся, что выбросами будут считаться поездки, длительность которых превышает 24 часа. 

Чуть сложнее с анализом поездок, длительность которых слишком мала. Потому что к ним относятся действительно реальные поездки на короткие расстояния, поездки, которые были отменены через секунду после того как включился счетчик, а также “телепортации” - перемещение на большие расстояния за считанные секунды. 
Условимся, что мы будем считать выбросами только последнюю группу. Как же нам их обнаружить наиболее простым способом?

Можно воспользоваться информацией о кратчайшем расстоянии, которое проезжает такси. Вычислить среднюю скорость автомобиля на кратчайшем пути следующим образом: 
$$avg\_speed= \frac{total\_distance}{1000*trip\_duration}*3600$$
Если мы построим диаграмму рассеяния средней скорости движения автомобилей, мы увидим следующую картину:


In [ ]:
avg_speed = taxi_data['total_distance'] / taxi_data['trip_duration'] * 3.6
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(x=avg_speed.index, y=avg_speed, ax=ax)
ax.set_xlabel('Index')
ax.set_ylabel('Average speed');

Как раз отсюда мы видим, что у нас есть “поездки-телепортации”, для которых средняя скорость более 1000 км/ч. Даже есть такая, средняя скорость которой составляла более 12000 км/ч! 

Давайте условимся, что предельная средняя скорость, которую могут развивать таксисты будет 300 км/ч. 


### Задание 2.11.
Найдите поездки, длительность которых превышает 24 часа. И удалите их из набора данных.

а) Сколько выбросов по признаку длительности поездки вам удалось найти?

Найдите поездки, средняя скорость которых по кратчайшему пути превышает 300 км/ч и удалите их из данных. 

б) Сколько выбросов по признаку скорости вам удалось найти?

In [17]:
long_trips = taxi_data['trip_duration'] > 24*3600

print(f'Количество поездок, длительность которых превышает 24 часа: {taxi_data[long_trips].shape[0]}')

taxi_data = taxi_data.drop(taxi_data[long_trips].index)

Количество поездок, длительность которых превышает 24 часа: 4


In [18]:
high_speed_trips = taxi_data['total_distance'] / taxi_data['trip_duration'] * 3.6 > 300

print(f'Количество поездок, средняя скорость которых по кратчайшему пути превышает 300 км/ч: {taxi_data[high_speed_trips].shape[0]}')

taxi_data = taxi_data.drop(taxi_data[high_speed_trips].index)

Количество поездок, средняя скорость которых по кратчайшему пути превышает 300 км/ч: 407


## 3. Разведывательный анализ данных (EDA)

В этой части нашего проекта мы с вами:
* Исследуем сформированный набор данных; 
* Попробуем найти закономерности, позволяющие сформулировать предварительные гипотезы относительно того, какие факторы являются решающими в определении длительности поездки;
* Дополним наш анализ визуализациями, иллюстрирующими; исследование. Постарайтесь оформлять диаграммы с душой, а не «для галочки»: навыки визуализации полученных выводов обязательно пригодятся вам в будущем.


Начинаем с целевого признака. Забегая вперед, скажем, что основной метрикой качества решения поставленной задачи будет RMSLE - Root Mean Squared Log Error, которая вычисляется на основе целевой переменной в логарифмическом масштабе. В таком случае целесообразно сразу логарифмировать признак длительности поездки и рассматривать при анализе логарифм в качестве целевого признака:
$$trip\_duration\_log = log(trip\_duration+1),$$
где под символом log подразумевается натуральный логарифм.


In [19]:
taxi_data['trip_duration_log'] = np.log(taxi_data['trip_duration']+1)

### Задание 3.1.
Постройте гистограмму и коробчатую диаграмму длительности поездок в логарифмическом масштабе (trip_duration_log). 
Исходя из визуализации, сделайте предположение, является ли полученное распределение нормальным? 
Проверьте свою гипотезу с помощью теста Д’Агостино при уровне значимости $\alpha=0.05$. 

а) Чему равен вычисленный p-value? Ответ округлите до сотых.

б) Является ли распределение длительности поездок в логарифмическом масштабе нормальным?

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=taxi_data['trip_duration_log'], marker_color='#007dff', name='', nbinsx=200))

fig.update_traces(hoverinfo='all', hovertemplate='trip_duration_log: %{x}<br>value: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of trip_duration_log (LOG)', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='trip_duration_log',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='value',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(x=taxi_data['trip_duration_log'], marker_color='#007dff', name=''))

fig.update_traces(hoverinfo='all', hovertemplate='trip_duration_log: %{x}<br>value: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of trip_duration_log (LOG)', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='trip_duration_log',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='value',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

In [22]:
# установка гипотез (нулевой и альтернативной)
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
# установка уровня значимости
alpha = 0.05

stat, p = stats.normaltest(taxi_data['trip_duration_log'])
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

# bнтерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

Statistics=138350.166, p-value=0.000
Данные не распределены нормально (мы отвергаем H0)


### Задание 3.2.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от таксопарка (vendor_id). 

Сравните два распределения между собой.

In [ ]:
vendor_1 = taxi_data['vendor_id'] == 1
vendor_2 = taxi_data['vendor_id'] == 2

fig = go.Figure()

fig.add_trace(go.Histogram(x=taxi_data[vendor_1]['trip_duration_log'], name='vendor_1', marker_color='#007dff', nbinsx=100))
fig.add_trace(go.Histogram(x=taxi_data[vendor_2]['trip_duration_log'], name='vendor_2', marker_color='#e75300', nbinsx=100))

fig.update_traces(hoverinfo="all", hovertemplate="trip_duration_log: %{x}<br>value: %{y}")

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of trip_duration_log on vendors', 
                  title_x = 0.5, title_font_size=25,
                  legend_orientation="h", legend=dict(x=.2, xanchor="center"),
                  height=600)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='trip_duration_log',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='value',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(x=taxi_data[vendor_1]['trip_duration_log'], name='vendor_1', marker_color='#007dff'))
fig.add_trace(go.Box(x=taxi_data[vendor_2]['trip_duration_log'], name='vendor_2', marker_color='#e75300'))

fig.update_traces(hoverinfo="all", hovertemplate="trip_duration_log: %{x}<br>value: %{y}")

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of trip_duration_log on vendors', 
                  title_x = 0.5, title_font_size=25,
                  legend_orientation="h", legend=dict(x=.2, xanchor="center"),
                  height=600)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='trip_duration_log',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='value',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

Распределения в группах практически не отличаются, признак vendor_id не имеет значения при определении длительности поездки.

### Задание 3.3.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от признака отправки сообщения поставщику (store_and_fwd_flag). 

Сравните два распределения между собой.

In [ ]:
flag_N = taxi_data['store_and_fwd_flag'] == 'N'
flag_Y = taxi_data['store_and_fwd_flag'] == 'Y'

fig = go.Figure()

fig.add_trace(go.Histogram(x=taxi_data[flag_N]['trip_duration_log'], name='flag_N', marker_color='#007dff', nbinsx=100))
fig.add_trace(go.Histogram(x=taxi_data[flag_Y]['trip_duration_log'], name='flag_Y', marker_color='#e75300', nbinsx=100))

fig.update_traces(hoverinfo="all", hovertemplate="trip_duration_log: %{x}<br>value: %{y}")

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of trip_duration_log on store_and_fwd_flag', 
                  title_x = 0.5, title_font_size=25,
                  legend_orientation="h", legend=dict(x=.2, xanchor="center"),
                  height=600)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='trip_duration_log',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='value',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

Распределения в группах значительно отличаются, признак store_and_fwd_flag имеет значение при определении длительности поездки.

### Задание 3.4.
Постройте две визуализации:
* Распределение количества поездок в зависимости от часа дня;
* Зависимость медианной длительности поездки от часа дня.

На основе построенных графиков ответьте на следующие вопросы:

а) В какое время суток такси заказывают реже всего?

б) В какое время суток наблюдается пик медианной длительности поездок?

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=taxi_data['pickup_hour'], marker_color='#007dff', name=''))

fig.update_traces(hoverinfo='all', hovertemplate='hour of the day: %{x}<br>number of trips: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of the number of trips from hour of the day', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='hour of the day',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='number of trips',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

In [ ]:
data = taxi_data.groupby('pickup_hour')['trip_duration'].median()

fig = go.Figure()

fig.add_trace(go.Bar(x=data.index, y=data, marker_color='#007dff', name=''))

fig.update_traces(hoverinfo='all', hovertemplate='hour of the day: %{x}<br>median number of trips: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of the median number of trips from hour of the day', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='hour of the day',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='median number of trips',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

Такси заказывают реже всего с 0 до 5 часов.

Пик медианной длительности поездок наблюдается с 13 до 18 часов.

### Задание 3.5.
Постройте две визуализации:
* Распределение количества поездок в зависимости от дня недели;
*  Зависимость медианной длительности поездки от дня недели.

На основе построенных графиков ответьте на следующие вопросы:
а) В какой день недели совершается больше всего поездок?
б) В какой день недели медианная длительность поездок наименьшая?


In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=taxi_data['pickup_day_of_week'], marker_color='#007dff', name=''))

fig.update_traces(hoverinfo='all', hovertemplate='day of week: %{x}<br>number of trips: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of the number of trips from the day of week', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='day of week',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='number of trips',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

In [ ]:
data = taxi_data.groupby('pickup_day_of_week')['trip_duration'].median()

fig = go.Figure()

fig.add_trace(go.Bar(x=data.index, y=data, marker_color='#007dff', name=''))

fig.update_traces(hoverinfo='all', hovertemplate='day of week: %{x}<br>median number of trips: %{y}')

fig.update_layout(margin=dict(l=20, r=20, t=90, b=10), 
                  title='Distribution of the median number of trips from the day of week', 
                  title_x = 0.5, title_font_size=20,
                  bargap=0.2,
                  showlegend=False)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell',), 
                 title_text='day of week',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='median number of trips',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

Больше всего поездок совершается в пятницу.

Медианная длительность поездок наименьшая в воскресенье.

### Задание 3.6.
Посмотрим на обе временные характеристики одновременно. 

Постройте сводную таблицу, по строкам которой отложены часы (pickup_hour), по столбцам - дни недели (pickup_day_of_week), а в ячейках - медианная длительность поездки (trip_duration). 

Визуализируйте полученную сводную таблицу с помощью тепловой карты (рекомендуемая палитра - coolwarm).

In [ ]:
data_pivot = pd.pivot_table(
    data=taxi_data,
    index='pickup_hour',
    columns='pickup_day_of_week',
    values='trip_duration',
    aggfunc='median'
)

fig = go.Figure(data=go.Heatmap(
                   x=data_pivot.columns,
                   y=data_pivot.index,
                   z=data_pivot,
                   hoverongaps = True,
                   name=''))

fig.update_traces(hoverinfo="all", 
                  hovertemplate="Day of week: %{x}<br>Hour of the day: %{y}<br>median trip duration: %{z}")

fig.update_layout(margin=dict(l=20, r=20, t=90, b=20), 
                  title='Dependence of the median trip duration on the day of the week and the hour of the day', 
                  title_x = 0.5, title_font_size=16,
                  legend_orientation="h", legend=dict(x=.5, xanchor="center"),
                  height=600)

fig.update_xaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='Day of week',
                 tickfont=dict(family='Rockwell', size=14))
fig.update_yaxes(title_font=dict(size=16, family='Rockwell'), 
                 title_text='Hour of the day',
                 tickfont=dict(family='Rockwell', size=14))

fig.show()

### Задание 3.7.
Постройте две диаграммы рассеяния (scatter-диаграммы):
* первая должна иллюстрировать географическое расположение точек начала поездок (pickup_longitude, pickup_latitude) 
* вторая должна географическое расположение точек завершения поездок (dropoff_longitude, dropoff_latitude).

Для этого на диаграммах по оси абсцисс отложите широту (longitude), а по оси ординат - долготу (latitude). 
Включите в визуализацию только те точки, которые находятся в пределах Нью-Йорка - добавьте следующие ограничения на границы осей абсцисс и ординат:
 
city_long_border = (-74.03, -73.75)

city_lat_border = (40.63, 40.85)

Добавьте на диаграммы расцветку по десяти географическим кластерам (geo_cluster), которые мы сгенерировали ранее. 

**Рекомендация:** для наглядности уменьшите размер точек на диаграмме рассеяния.  


In [ ]:
# ограничения по координатам
long = (taxi_data['pickup_longitude'] >= -74.03) & (taxi_data['pickup_longitude'] <= -73.75)
lat = (taxi_data['pickup_latitude'] >= 40.63) & (taxi_data['pickup_latitude'] <= 40.85)
# таблица с ограничениями по координатам
data = taxi_data[long & lat]

fig = make_subplots(rows=1, cols=2, 
                    x_title='pickup_longitude',
                    y_title='pickup_latitude',
                    subplot_titles=('Coordinates of the start of trips', 'Coordinates of the end of trips'))

clusters = set(data['geo_cluster'].values)

for i in clusters:
    fig.add_trace(go.Scattergl(x = data[data['geo_cluster'] == i]['pickup_longitude'],
                               y = data[data['geo_cluster'] == i]['pickup_latitude'],
                               name='geo_claster_' + str(i),
                               mode = 'markers',
                               marker=dict(size=1)),
                  1, 1)

for i in clusters:
    fig.add_trace(go.Scattergl(x = data[data['geo_cluster'] == i]['pickup_longitude'],
                               y = data[data['geo_cluster'] == i]['pickup_latitude'],
                               name='geo_claster_' + str(i),
                               mode = 'markers',
                               marker=dict(size=1)),
                  1, 2)
    
fig.show()

## 4. Отбор и преобразование признаков

Перед тем как перейти к построению модели, осталось сделать ещё несколько шагов.
* Следует помнить, что многие алгоритмы машинного обучения не могут обрабатывать категориальные признаки в их обычном виде. Поэтому нам необходимо их закодировать;
* Надо отобрать признаки, которые мы будем использовать для обучения модели;
*  Необходимо масштабировать и трансформировать некоторые признаки для того, чтобы улучшить сходимость моделей, в основе которых лежат численные методы.


In [32]:
print('Shape of data: {}'.format(taxi_data.shape))
print('Columns: {}'.format(taxi_data.columns))

Shape of data: (1458233, 27)
Columns: Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_date', 'pickup_hour', 'pickup_day_of_week',
       'pickup_holiday', 'total_distance', 'total_travel_time',
       'number_of_steps', 'haversine_distance', 'direction', 'geo_cluster',
       'temperature', 'visibility', 'wind speed', 'precip', 'events',
       'trip_duration_log'],
      dtype='object')


Для удобства работы сделаем копию исходной таблицы с поездками:

In [33]:
train_data = taxi_data.copy()
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,number_of_steps,haversine_distance,direction,geo_cluster,temperature,visibility,wind speed,precip,events,trip_duration_log
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,5.0,1.498521,99.970196,0,4.4,8.0,27.8,0.3,None,6.122493
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,6.0,1.805507,-117.153768,6,28.9,16.1,7.4,0.0,None,6.498282
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,16.0,6.385098,-159.680165,6,-6.7,16.1,24.1,0.0,None,7.661527
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,4.0,1.485498,-172.737700,6,7.2,16.1,25.9,0.0,None,6.063785
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,5.0,1.188588,179.473585,0,9.4,16.1,9.3,0.0,None,6.077642


### Задание 4.1.
Сразу позаботимся об очевидных неинформативных и избыточных признаках. 

а) Какой из признаков является уникальным для каждой поездки и не несет полезной информации в определении ее продолжительности?

б) Утечка данных (data leak) - это…

в) Подумайте, наличие какого из признаков в обучающем наборе данных создает утечку данных?

г) Исключите выбранные в пунктах а) и в) признаки из исходной таблицы с данными. Сколько столбцов в таблице у вас осталось?


In [34]:
print('Уникальным признаком для каждой поездки, который не несет полезной информации в определении ее продолжительности, явлется признак id.')
print('Утечка данных (data leak) - это ситуация, в которой данные используемые для обучения модели, содержат прямую или косвенную информацию о целевой переменной, но эти данные недоступны в реальных условиях.')
print('Наличие признака dropoff_datetime в обучающем наборе данных создает утечку данных.')
train_data.drop(['id', 'dropoff_datetime'], axis=1, inplace=True)
print('Осталось 25 признаков.')

Уникальным признаком для каждой поездки, который не несет полезной информации в определении ее продолжительности, явлется признак id.
Утечка данных (data leak) - это ситуация, в которой данные используемые для обучения модели, содержат прямую или косвенную информацию о целевой переменной, но эти данные недоступны в реальных условиях.
Наличие признака dropoff_datetime в обучающем наборе данных создает утечку данных.
Осталось 25 признаков.


Ранее мы извлекли всю необходимую для нас информацию из даты начала поездки, теперь мы можем избавиться от этих признаков, так как они нам больше не понадобятся:


In [35]:
drop_columns = ['pickup_datetime', 'pickup_date']
train_data = train_data.drop(drop_columns, axis=1)
print('Shape of data:  {}'.format(train_data.shape))

Shape of data:  (1458233, 23)


### Задание 4.2.

Закодируйте признак vendor_id в таблице train_data таким образом, чтобы он был равен 0, если идентификатор таксопарка равен 1, и 1 — в противном случае.

Закодируйте признак store_and_fwd_flag в таблице train_data таким образом, чтобы он был равен 0, если флаг выставлен в значение 'N', и 1 — в противном случае.

а) Рассчитайте среднее по закодированному столбцу vendor_id. Ответ приведите с точностью до сотых.

б) Рассчитайте среднее по закодированному столбцу store_and_fwd_flag. Ответ приведите с точностью до тысячных.



In [36]:
train_data['vendor_id'] = train_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)

print(f'Среднее по закодированному столбцу vendor_id: {round(train_data["vendor_id"].mean(), 2)}')
print(f'Среднее по закодированному столбцу store_and_fwd_flag: {round(train_data["store_and_fwd_flag"].mean(), 3)}')

Среднее по закодированному столбцу vendor_id: 0.53
Среднее по закодированному столбцу store_and_fwd_flag: 0.006


### Задание 4.3.
Создайте таблицу data_onehot из закодированных однократным кодированием признаков pickup_day_of_week, geo_cluster и events в таблице train_data с помощью OneHotEncoder из библиотеки sklearn. Параметр drop выставите в значение 'first', чтобы удалять первый бинарный столбец, тем самым не создавая излишних признаков.

В результате работы OneHotEncoder вы получите безымянный numpy-массив, который нам будет необходимо преобразовать обратно в DataFrame, для более удобной работы в дальнейшем. Чтобы получить имена закодированных столбцов у объекта типа OneHotEncoder есть специальный метод get_feature_names_out(). Он возвращает список новых закодированных имен столбцов в формате <оригинальное имя столбца>_<имя категории>.

Пример использования:

``` python
# Получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()
# Составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns=column_names)
```

В этом псевдокоде:
* one_hot_encoder - объект класса OneHotEncoder
* data_onehot - numpy-массив, полученный в результате трансформации кодировщиком

В результате выполнения задания у вас должен быть образован DataFrame `data_onehot`, который содержит кодированные категориальные признаки pickup_day_of_week, geo_cluster и events. 


Сколько бинарных столбцов у вас получилось сгенерировать с помощью однократного кодирования?


In [37]:
# создаем объект класса OneHotEncoder, с удалением оригинального признака
one_hot_encoder = preprocessing.OneHotEncoder(drop='first') 
# список признаков для кодирования
columns_to_change = ['pickup_day_of_week', 'geo_cluster', 'events']

# одновременно обучаем кодировщик и кодируем данные с переводом результата в массив numpy
data_onehot = one_hot_encoder.fit_transform(train_data[columns_to_change]).toarray()

# получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()

# составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns = column_names)

print(f'Количество новых бинарных столбцов: {data_onehot.shape[1]}')

Количество новых бинарных столбцов: 18


Добавим полученную таблицу с закодированными признаками:

In [38]:
train_data = pd.concat(
    [train_data.reset_index(drop=True).drop(columns_to_change, axis=1), data_onehot], 
    axis=1
)
print(f'Shape of data: {train_data.shape}')

Shape of data: (1458233, 38)


Теперь, когда категориальные признаки предобработаны, сформируем матрицу наблюдений X, вектор целевой переменной y и его логарифм y_log. В матрицу наблюдений войдут все столбцы из таблицы с поездками за исключением целевого признака trip_duration и его логарифмированной версии trip_duration_log:


In [39]:
X = train_data.drop(['trip_duration', 'trip_duration_log'], axis=1)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

Все наши модели мы будем обучать на логарифмированной версии y_log. 

Выбранный тип валидации - hold-out. Разобьем выборку на обучающую и валидационную в соотношении 67/33:

In [40]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X, y_log, 
    test_size=0.33, 
    random_state=42
)

На данный момент у нас достаточно много признаков: скорее всего, не все из них будут важны. Давайте оставим лишь те, которые сильнее всего связаны с целевой переменной и точно будут вносить вклад в повышение качества модели.


### Задание 4.4.
С помощью SelectKBest отберите 25 признаков, наилучшим образом подходящих для предсказания целевой переменной в логарифмическом масштабе. Отбор реализуйте по обучающей выборке, используя параметр score_func = f_regression.

Укажите признаки, которые вошли в список отобранных


In [41]:
# отбираем 25 признаков
selector = feature_selection.SelectKBest(
    score_func = feature_selection.f_regression,
    k = 25
)
selector.fit(X_train, y_train_log)

# оставляем только полученные 25 признаков
best_features = selector.get_feature_names_out()
X_train = X_train[best_features]
X_valid = X_valid[best_features]
print('Признаки, которые вошли в список отобранных:\n', best_features)

Признаки, которые вошли в список отобранных:
 ['vendor_id' 'passenger_count' 'pickup_longitude' 'pickup_latitude'
 'dropoff_longitude' 'dropoff_latitude' 'store_and_fwd_flag' 'pickup_hour'
 'pickup_holiday' 'total_distance' 'total_travel_time' 'number_of_steps'
 'haversine_distance' 'temperature' 'pickup_day_of_week_Monday'
 'pickup_day_of_week_Saturday' 'pickup_day_of_week_Sunday'
 'pickup_day_of_week_Thursday' 'pickup_day_of_week_Tuesday'
 'pickup_day_of_week_Wednesday' 'geo_cluster_1' 'geo_cluster_4'
 'geo_cluster_5' 'geo_cluster_7' 'geo_cluster_8']


Так как мы будем использовать различные модели, в том числе внутри которых заложены численные методы оптимизации, то давайте заранее позаботимся о масштабировании факторов. 


### Задание 4.5.
Нормализуйте предикторы в обучающей и валидационной выборках с помощью MinMaxScaler из библиотеки sklearn. Помните, что обучение нормализатора производится на обучающей выборке, а трансформация на обучающей и валидационной!

Рассчитайте среднее арифметическое для первого предиктора (т. е. для первого столбца матрицы) из валидационной выборки. Ответ округлите до сотых.


In [42]:
# инициализируем нормализатор MinMaxScaler
scaler = preprocessing.MinMaxScaler()

# преобразовываем данные
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

# преобразуем массивы в DataFrame
X_train = pd.DataFrame(X_train, columns = best_features)
X_valid = pd.DataFrame(X_valid, columns = best_features)

print('Среднее арифметическое для первого предиктора из валидационной выборки:',
      round(X_valid.iloc[:, [0]].mean(), 2))

Среднее арифметическое для первого предиктора из валидационной выборки: vendor_id    0.54
dtype: float64


## 5. Решение задачи регрессии: линейная регрессия и деревья решений

Определим метрику, по которой мы будем измерять качество наших моделей. Мы будем следовать канонам исходного соревнования на Kaggle и в качестве метрики использовать RMSLE (Root Mean Squared Log Error), которая вычисляется как:
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(log(y_i+1)-log(\hat{y_i}+1))^2},$$
где:
* $y_i$ - истинная длительность i-ой поездки на такси (trip_duration)
* $\hat{y_i}$- предсказанная моделью длительность i-ой поездки на такси

Заметим, что логарифмирование целевого признака мы уже провели заранее, поэтому нам будет достаточно вычислить метрику RMSE для модели, обученной прогнозировать длительность поездки такси в логарифмическом масштабе:
$$z_i=log(y_i+1),$$
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(z_i-\hat{z_i})^2}=\sqrt{MSE(z_i,\hat{z_i})}$$ 



### Задание 5.1.
Постройте модель линейной регрессии на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [43]:
# создаём объект класса LinearRegression
lr_model = linear_model.LinearRegression()

# обучаем модель
lr_model.fit(X_train, y_train_log)

# делаем предсказания
y_train_pred = lr_model.predict(X_train)
y_valid_pred = lr_model.predict(X_valid)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))

RMSE на тренировочной выборке: 0.53
RMSE на валидационной выборке: 0.54


### Задание 5.2.
Сгенерируйте полиномиальные признаки 2-ой степени с помощью PolynomialFeatures из библиотеки sklearn. Параметр include_bias выставите в значение False.

Постройте модель полиномиальной регрессии 2-ой степени на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [44]:
# создаем объект класса PolynomialFeatures и 
# генератор полиномиальных признаков
poly = preprocessing.PolynomialFeatures(2, include_bias=False)
poly.fit(X_train)

# генерируем полиномиальные признаки
X_train_poly = poly.transform(X_train)
X_valid_poly = poly.transform(X_valid)

# строим модель полиномиальной регрессии и 
# делаем предсказания
lr_poly = linear_model.LinearRegression()
lr_poly.fit(X_train_poly, y_train_log)
y_train_pred = lr_poly.predict(X_train_poly)
y_valid_pred = lr_poly.predict(X_valid_poly)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))


RMSE на тренировочной выборке: 0.47
RMSE на валидационной выборке: 0.61


Да, у модели наблюдаются признаки переобучения.

### Задание 5.3.
Постройте модель полиномиальной регрессии 2-ой степени с L2-регуляризацией (регуляризация по Тихонову) на обучающей выборке  (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Коэффициент регуляризации $\alpha$ установите равным 1, остальные параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [45]:
# создаём объект класса LinearRegression с L2-регуляризацией
lr_ridge = linear_model.Ridge(alpha=1)

# обучаем модель
lr_ridge.fit(X_train_poly, y_train_log)

# делаем предсказания
y_train_pred = lr_ridge.predict(X_train_poly)
y_valid_pred = lr_ridge.predict(X_valid_poly)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))

RMSE на тренировочной выборке: 0.48
RMSE на валидационной выборке: 0.48


### Задание 5.4.
Постройте модель дерева решений (DecisionTreeRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию. 

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [46]:
# создаём объект класса DecisionTreeRegressor и 
# обучаем модель
dt_reg = tree.DecisionTreeRegressor(random_state = 42)
dt_reg.fit(X_train, y_train_log)

# делаем предсказания
y_train_pred = dt_reg.predict(X_train)
y_valid_pred = dt_reg.predict(X_valid)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))

RMSE на тренировочной выборке: 0.0
RMSE на валидационной выборке: 0.56


Да, у модели наблюдаются признаки переобучения.

### Задание 5.5.
Переберите все возможные варианты глубины дерева решений в диапазоне от 7 до 20:

max_depths = range(7, 20)

Параметр random_state задайте равным 42.

Постройте линейные графики изменения метрики RMSE на тренировочной и валидационной выборках в зависимости от значения параметра глубины дерева решений. 

а) Найдите оптимальное значение максимальной глубины дерева, для которой будет наблюдаться минимальное значение RMSLE на обучающей выборке, но при этом еще не будет наблюдаться переобучение (валидационная кривая еще не начинает возрастать).

б) Чему равно значение метрик RMSLE на тренировочной и валидационной выборках для дерева решений с выбранной оптимальной глубиной? Ответ округлите до сотых.


In [ ]:
# список значений глубины дерева решений
max_depths = range(7, 20)

# создаём пустые списки для результатов
train_scores = []
valid_scores = []

for depth in max_depths:
    # создаём и обучаем модель
    dt_reg = tree.DecisionTreeRegressor(max_depth = depth, random_state = 42)
    dt_reg.fit(X_train, y_train_log)
    # делаем предсказания 
    y_train_pred = dt_reg.predict(X_train)
    y_valid_pred = dt_reg.predict(X_valid)
    # рассчитываем RMSLE для двух выборок и добавляем в списки
    train_scores.append(
        np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred))
        )
    valid_scores.append(
        np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred))
        )
    
fig = go.Figure()

fig.add_trace(go.Scatter(
                         x=list(max_depths),
                         y=train_scores,
                         mode='lines+markers',
                         name='train'
                         )
              )
             
             

fig.add_trace(go.Scatter(
                         x=list(max_depths),
                         y=valid_scores,
                         mode='lines+markers',
                         name='valid'
                         )
              )
    
fig.update_traces(hoverinfo='all', 
                  hovertemplate='max_depths: %{x}<br>RMSLE: %{y}')

fig.update_layout(title='Dependence of RMSLE on max_depths', 
                  title_x = 0.5, title_font_size=20,
                  legend_orientation='h', legend=dict(x=.5, xanchor='center'),
                  height=600)

fig.update_xaxes(title_text='max_depths')
fig.update_yaxes(title_text='RMSLE')

fig.show()    

# извлекаем индекс лучшего RMSLE на валидационной выборке и
# определяем оптимальную глубину дерева и значения RMSLE
best_index = valid_scores.index(min(valid_scores))
print('Оптимальная глубина дерева решений:', max_depths[best_index])
print('RMSLE на тренировочной выборке:', round(train_scores[best_index], 2))
print('RMSLE на валидационной выборке:', round(valid_scores[best_index], 2))

## 6. Решение задачи регрессии: ансамблевые методы и построение прогноза

Переходим к тяжелой артиллерии: ансамблевым алгоритмам. 

### Задание 6.1.

Постройте модель случайного леса на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* n_estimators=200,
* max_depth=12,
* criterion='squared_error',
* min_samples_split=20,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [48]:
# создаём объект класса RandomForestRegressor
rf_reg = ensemble.RandomForestRegressor(
    n_estimators = 200,
    max_depth = 12,
    criterion = 'squared_error',
    min_samples_split = 20,
    random_state = 42,
    n_jobs = -1
)

# обучаем модель
rf_reg.fit(X_train, y_train_log)

# делаем предсказания
y_train_pred = rf_reg.predict(X_train)
y_valid_pred = rf_reg.predict(X_valid)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))

RMSE на тренировочной выборке: 0.4
RMSE на валидационной выборке: 0.41


### Задание 6.2.
Постройте модель градиентного бустинга над деревьями решений (GradientBoostingRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* learning_rate=0.5,
* n_estimators=100,
* max_depth=6, 
* min_samples_split=30,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [49]:
# создаём объект класса GradientBoostingRegressor
gb_reg = ensemble.GradientBoostingRegressor(
    learning_rate = 0.5,
    n_estimators = 100,
    max_depth = 6,
    min_samples_split = 30,
    random_state = 42
)

# обучаем модель
gb_reg.fit(X_train, y_train_log)

# делаем предсказания
y_train_pred = gb_reg.predict(X_train)
y_valid_pred = gb_reg.predict(X_valid)

# рассчитываем RMSLE
print('RMSE на тренировочной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_train_log, y_train_pred)), 2))
print('RMSE на валидационной выборке:',
    round(np.sqrt(metrics.mean_squared_error(y_valid_log, y_valid_pred)), 2))

RMSE на тренировочной выборке: 0.37
RMSE на валидационной выборке: 0.39


### Задание 6.3.
Какая из построенных вами моделей показала наилучший результат (наименьшее значение RMSLE на валидационной выборке)?
* Линейная регрессия
* Полиномиальная регрессия 2ой степени
* Дерево решений
* Случайный лес
* Градиентный бустинг над деревьями решений


Градиентный бустинг над деревьями решений показал наилучший результат (наименьшее значение RMSLE на валидационной выборке.

### Задание 6.4.
Постройте столбчатую диаграмму коэффициентов значимости каждого из факторов.

Укажите топ-3 наиболее значимых для предсказания целевого признака - длительности поездки в логарифмическом масштабе - факторов.


In [ ]:
features = X_train.columns                # признаки
importances = gb_reg.feature_importances_ # важность признаков

fig = go.Figure()

fig.add_trace(go.Bar(x=features, y=importances, name='',  marker_color=[i for i in range(1, 26)]))

fig.update_traces(hoverinfo='all', hovertemplate='feature: %{x}<br>mportances: %{y}')

fig.update_layout(title='Bar plot feature importances', 
                  title_x = 0.5, title_font_size=25,
                  bargap=0.2,
                  showlegend=False,
                  height=600)

fig.update_xaxes(tickangle=90)

fig.update_yaxes(title_text='importances')

fig.show()

### Задание 6.5.
Для лучшей из построенных моделей рассчитайте медианную абсолютную ошибку (MeAE - в sklearn функция median_absolute_error) предсказания длительности поездки такси на валидационной выборке:
$$ MeAE = median(|y_i-\hat{y_i}|)$$

Значение метрики MeAE переведите в минуты и округлите до десятых.


In [51]:
# переводим из логарифимического маштаба в изначальный
y_valid_exp = np.exp(y_valid_log) - 1
y_valid_pred_exp = np.exp(y_valid_pred) - 1

# рассчитываем медианную абсолютную ошибку
meae_valid = metrics.median_absolute_error(y_valid_exp, y_valid_pred_exp)

# переводим значение метрики в минуты и округляем
print('MeAE на валидационной выборке:', round(meae_valid / 60, 1), 'мин')

MeAE на валидационной выборке: 1.8 мин


Финальный шаг - сделать submit -  предсказание для отложенного тестового набора данных. 

Прочитаем тестовые данные и заранее выделим столбец с идентификаторами поездок из тестового набора данных. Он нам еще пригодится:


In [52]:
test_data = pd.read_csv("data/test.csv")
osrm_data_test = pd.read_csv("data/osrm_data_test.csv")
test_id = test_data['id']

Перед созданием прогноза для тестовой выборки необходимо произвести все манипуляции с данными, которые мы производили с тренировочной выборкой, а именно:
* Перевести признак pickup_datetime в формат datetime;
* Добавить новые признаки (временные, географические, погодные и другие факторы);
* Произвести очистку данных от пропусков;
* Произвести кодировку категориальных признаков:
    * Закодировать бинарные признаки;
    * Закодировать номинальные признаки с помощью обученного на тренировочной выборке OneHotEncoder’а;
* Сформировать матрицу наблюдений, оставив в таблице только те признаки, которые были отобраны с помощью SelectKBest;
* Нормализовать данные с помощью обученного на тренировочной выборке MinMaxScaler’а.


In [53]:
test_data['pickup_datetime']=pd.to_datetime(test_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
test_data = add_datetime_features(test_data)
test_data = add_holiday_features(test_data, holiday_data)
test_data = add_osrm_features(test_data, osrm_data_test)
test_data = add_geographical_features(test_data)
test_data = add_cluster_features(test_data, kmeans)
test_data = add_weather_features(test_data, weather_data)
test_data = fill_null_weather_data(test_data)

test_data['vendor_id'] = test_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)
test_data_onehot = one_hot_encoder.fit_transform(test_data[columns_to_change]).toarray()
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
test_data_onehot = pd.DataFrame(test_data_onehot, columns=column_names)

test_data = pd.concat(
    [test_data.reset_index(drop=True).drop(columns_to_change, axis=1), test_data_onehot], 
    axis=1
)
X_test = test_data[best_features]
X_test_scaled = scaler.transform(X_test)
print('Shape of data: {}'.format(X_test.shape))

Shape of data: (625134, 25)


Только после выполнения всех этих шагов можно сделать предсказание длительности поездки для тестовой выборки. Не забудьте перевести предсказания из логарифмического масштаба в истинный, используя формулу:
$$y_i=exp(z_i)-1$$

После того, как вы сформируете предсказание длительности поездок на тестовой выборке вам необходимо будет создать submission-файл в формате csv, отправить его на платформу Kaggle и посмотреть на результирующее значение метрики RMSLE на тестовой выборке.

Код для создания submission-файла:


In [54]:
# делаем предсказание для тестовой выборки
y_test_pred_log = gb_reg.predict(X_test_scaled)

# переводим предсказания из логарифмического маштаба в истинный
y_test_predict = np.exp(y_test_pred_log) - 1

# создаём submission-файл
submission = pd.DataFrame({'id': test_id, 'trip_duration': y_test_predict})
submission.to_csv('data/submission_gb.csv', index=False)

c:\Users\artem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names



### **В качестве бонуса**

В завершение по ансамблевым мы предлагаем вам попробовать улучшить свое предсказание, воспользовавшись моделью экстремального градиентного бустинга (XGBoost) из библиотеки xgboost.

**XGBoost** - современная модель машинного обучения, которая является продолжением идеи градиентного бустинга Фридмана. У нее есть несколько преимуществ по сравнению с классической моделью градиентного бустинга из библиотеки sklearn: повышенная производительность путем параллелизации процесса обучения, повышенное качество решения за счет усовершенствования алгоритма бустинга, меньшая склонность к переобучению и широкий функционал возможности управления параметрами модели.


Для ее использования необходимо для начала установить пакет xgboost:

In [55]:
# !pip install xgboost

После чего модуль можно импортировать:

In [56]:
import xgboost as xgb

Перед обучением модели необходимо перевести наборы данных в тип данных xgboost.DMatrix:

In [59]:
# создание матриц наблюдений в формате DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train_log, feature_names=best_features)
dvalid = xgb.DMatrix(X_valid, label=y_valid_log, feature_names=best_features)
dtest = xgb.DMatrix(X_test_scaled, feature_names=best_features)

Обучение модели XGBoost происходит с помощью метода train, в который необходимо передать параметры модели, набор данных, количество базовых моделей в ансамбле, а также дополнительные параметры:


In [60]:
# Гиперпараметры модели
xgb_pars = {'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.9, 
            'max_depth': 6, 'subsample': 0.9, 'lambda': 1, 'nthread': -1, 
            'booster' : 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:squarederror'
           }
# Тренировочная и валидационная выборка
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
# Обучаем модель XGBoost
model = xgb.train(
    params=xgb_pars, # гиперпараметры модели
    dtrain=dtrain, # обучающая выборка
    num_boost_round=300, # количество моделей в ансамбле
    evals=watchlist, # выборки, на которых считается матрица
    early_stopping_rounds=20, # раняя остановка
    maximize=False, # смена поиска максимума на минимум
    verbose_eval=10 # шаг, через который происходит отображение метрик
)

[0]	train-rmse:5.42218	valid-rmse:5.42189
[10]	train-rmse:1.93581	valid-rmse:1.93575
[20]	train-rmse:0.78514	valid-rmse:0.78601
[30]	train-rmse:0.47873	valid-rmse:0.48081
[40]	train-rmse:0.42070	valid-rmse:0.42344
[50]	train-rmse:0.40891	valid-rmse:0.41214
[60]	train-rmse:0.40378	valid-rmse:0.40737
[70]	train-rmse:0.40086	valid-rmse:0.40482
[80]	train-rmse:0.39864	valid-rmse:0.40286
[90]	train-rmse:0.39712	valid-rmse:0.40160
[100]	train-rmse:0.39578	valid-rmse:0.40051
[110]	train-rmse:0.39465	valid-rmse:0.39967
[120]	train-rmse:0.39360	valid-rmse:0.39897
[130]	train-rmse:0.39229	valid-rmse:0.39791
[140]	train-rmse:0.39106	valid-rmse:0.39707
[150]	train-rmse:0.38986	valid-rmse:0.39626
[160]	train-rmse:0.38893	valid-rmse:0.39563
[170]	train-rmse:0.38812	valid-rmse:0.39512
[180]	train-rmse:0.38739	valid-rmse:0.39466
[190]	train-rmse:0.38649	valid-rmse:0.39408
[200]	train-rmse:0.38573	valid-rmse:0.39370
[210]	train-rmse:0.38505	valid-rmse:0.39335
[220]	train-rmse:0.38464	valid-rmse:0.39322

Предсказать целевой признак на новых данных можно с помощью метода predict():

In [61]:
# делаем предсказание на тестовом наборе данных
y_test_predict = np.exp(model.predict(dtest)) - 1
print('Modeling RMSLE %.5f' % model.best_score)

Modeling RMSLE 0.39174


In [63]:
# создаём submission-файл
submission_xgb = pd.DataFrame({'id': test_id, 'trip_duration': y_test_predict})
submission_xgb.to_csv('data/submission_xgb.csv', index = False)

Также как и все модели, основанные на использовании деревьев решений в качестве базовых моделей, XGBoost имеет возможность определения коэффициентов важности факторов. Более того, в библиотеку встроена возможность визуализации важность факторов в виде столбчатой диаграммы. За эту возможность отвечает функция plot_importance():


In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
xgb.plot_importance(model, ax = ax, height=0.5)